# 📌 Understanding `Annotated` in Python

## 🔹 What is `Annotated`?
`Annotated` is a feature introduced in **Python 3.9** (backported in `typing_extensions` for older versions). It allows us to attach **metadata** to type hints, which can be used by tools like **Pydantic** or static type checkers to enforce additional constraints.

### ✅ Basic Syntax
```python
from typing import Annotated

CustomType = Annotated[int, "Some metadata"]
```

This does not change the actual type (int in this case) but adds extra metadata that external tools can use.

In [1]:
from typing import Annotated

def process_age(age: Annotated[int, "Age must be a positive integer"]) -> str:
    return f"Age: {age}"

print(process_age(25))  # Output: Age: 25

Age: 25


Here, the **metadata** `"Age must be a positive integer"` is added but does not enforce validation—it is just a hint for developers or tools.

# 🔍 Understanding `Annotated` in Python

The `Annotated` type hint in Python is a powerful tool that allows us to attach metadata, constraints, and validation logic to types. While many think `Annotated` is only for metadata, it can also be used with **Pydantic validation**, `Field`, and even custom functions.

---

## 🚀 Why Use `Annotated`?  

`Annotated` enhances type hints by allowing additional information that tools and libraries (like Pydantic) can use for validation, constraints, or documentation.

---

## 🔥 Beyond Metadata: Other Uses of `Annotated`

### 2 Using `Annotated` with `Field`  
In **Pydantic**, we can use `Field` to add validation and constraints.

```python
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    age: Annotated[int, Field(gt=18, description="Age must be greater than 18")]

# ✅ Valid: User(age=25)
# ❌ Invalid: User(age=17) -> Raises validation error

In [3]:
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    age: Annotated[int, Field(gt=0, lt=120)]  # Age must be between 1 and 119

user = User(age=25)  # ✅ Works fine

In [4]:
user = User(age=150)  # ❌ Raises validation error

ValidationError: 1 validation error for User
age
  Input should be less than 120 [type=less_than, input_value=150, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than

### 3. Using Annotated with Custom Functions

In [16]:
from typing import Annotated

def validate_age(value: int) -> int:
    if value < 0:
        raise ValueError("Age cannot be negative")
    return value

AgeType = Annotated[int, validate_age]  # ❌ This alone does nothing at runtime

def set_age(age: int) -> str:
    age = validate_age(age)  # ✅ Explicitly apply validation
    return f"User age is {age}"

print(set_age(-5))  # ❌ Raises ValueError: "Age cannot be negative"

ValueError: Age cannot be negative

🔥 Key Takeaways
✔ `Annotated` **does not enforce rules by itself** but provides metadata for validation tools.
✔ Can be used with `pydantic`, `Field`, or custom validation functions.
✔ Helps in making `type hints more descriptive` for documentation and static analysis.

In [ ]:
from typing import annotated_types
PositiveInt = Annotated[int, annotated_types.Gt(0)]


ImportError: cannot import name 'annotated_types' from 'typing' (C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\typing.py)

## 🔥 Using `Annotated` with `annotated_types`

`annotated_types` provides constraints like:

- **`Gt(value)`**: Greater than a specific value  
- **`Ge(value)`**: Greater than or equal to a value  
- **`Lt(value)`**: Less than a specific value  
- **`Le(value)`**: Less than or equal to a value  
- **`Interval(ge=a, le=b)`**: Restrict values within a range  

In [7]:
from typing import Annotated
from annotated_types import Gt  # Gt (Greater than) enforces a value > 0

PositiveInt = Annotated[int, Gt(0)]

def process_age(age: PositiveInt) -> str:
    return f"User age: {age}"

print(process_age(25))  # ✅ Works fine

User age: 25


In [8]:
print(process_age(-5))  # ❌ Raises an error if validated using a framework

User age: -5


In [10]:
from pydantic import BaseModel
from typing import Annotated
from annotated_types import Gt

class User(BaseModel):
    age: Annotated[int, Gt(0)]  # Ensures age is > 0

user = User(age=30)  # ✅ Works
user.age

30

In [11]:
user = User(age=-5)  # ❌ Raises validation error

ValidationError: 1 validation error for User
age
  Input should be greater than 0 [type=greater_than, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than

In [20]:
from typing import Annotated
from annotated_types import Gt, Le

PositiveInt = Annotated[int, Gt(0)]
LimitedFloat = Annotated[float, Gt(0), Le(100)]  # Value must be between 0 and 100
class Score(BaseModel):
    score : PositiveInt

Score(score=85.5)


ValidationError: 1 validation error for Score
score
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=85.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/int_from_float

In [21]:
from typing import Annotated
from annotated_types import Gt, Le
from pydantic import BaseModel

PositiveInt = Annotated[int, Gt(0)]
LimitedFloat = Annotated[float, Gt(0), Le(100)]  # Float between 0 and 100

class Score(BaseModel):
    score: LimitedFloat  # ✅ Accepts both int and float in range

print(Score(score=85.5))  # ✅ Works fine

score=85.5


In [22]:
print(Score(score=150))   # ❌ Raises ValidationError (out of range)
# print(Score(score=-5))    # ❌ Raises ValidationError (negative)

ValidationError: 1 validation error for Score
score
  Input should be less than or equal to 100 [type=less_than_equal, input_value=150, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal

## Pydantic Revision

In [24]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

user = User(**external_data)  

print(user.id)  
print(user.model_dump())  

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


If validation fails, Pydantic will raise an error with a breakdown of what was wrong:

In [26]:
# continuing the above example...

from datetime import datetime
from pydantic import BaseModel, PositiveInt, ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}  

try:
    User(**external_data)
except ValidationError as e:
    print(e.json(indent=2))  

[
  {
    "type": "int_parsing",
    "loc": [
      "id"
    ],
    "msg": "Input should be a valid integer, unable to parse string as an integer",
    "input": "not an int",
    "url": "https://errors.pydantic.dev/2.10/v/int_parsing"
  },
  {
    "type": "missing",
    "loc": [
      "signup_ts"
    ],
    "msg": "Field required",
    "input": {
      "id": "not an int",
      "tastes": {}
    },
    "url": "https://errors.pydantic.dev/2.10/v/missing"
  }
]
